In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html

In [2]:
r = requests.get("https://www.pba.com/players/jason-belmonte")

In [3]:
r

<Response [200]>

In [4]:
html         = lxml.html.fromstring(r.text)
core         = html.xpath('/html/body/div[1]/div/main/div/div[2]/div[2]/div/div/div[2]/article/div/div[4]/div[1]/div[2]/div/div/div/div/table')
tbody        = core[0].getchildren()
#div          = tbody[0].getchildren()

In [5]:
tbody

[<Element thead at 0x7fdd44412450>, <Element tbody at 0x7fdd444124a0>]

In [6]:
tbody[0].getchildren()[0].text_content()

'\n                                                Year\n                                                Events\n                                                Cashes\n                                                Match Play\n                                                CRA\n                                                Titles\n                                                AVG\n                                                Earnings\n              '

In [7]:
len(tbody[1])

13

In [8]:
data = {
    "Years":            [],
    "Events":           [],
    "Cashes":           [],
    "Match Play":       [],
    "CRA":              [],
    "Titles":           [],
    "AVG":              [],
    "Earnings":         [],
}

for row in tbody[1].getchildren()[1:13]:
    data["Years"].append(row.getchildren()[0].text_content().strip())
    data["Events"].append(row.getchildren()[1].text_content().strip())
    data["Cashes"].append(row.getchildren()[2].text_content().strip())
    data["Match Play"].append(row.getchildren()[3].text_content().strip())
    data["CRA"].append(row.getchildren()[4].text_content().strip())
    data["Titles"].append(row.getchildren()[5].text_content().strip())
    data["AVG"].append(row.getchildren()[6].text_content().strip())
    data["Earnings"].append(row.getchildren()[7].text_content().strip())
df = pd.DataFrame(data)

In [9]:
df

,Years,Events,Cashes,Match Play,CRA,Titles,AVG,Earnings
0,2020,9,9,7,5,2,226.790,"$232,300.00"
1,2019,19,17,15,12,4,225.620,"$285,290.00"
2,2018,14,10,6,3,2,221.380,"$110,500.00"
3,2017,15,15,11,7,4,229.390,"$238,912.00"
4,2016,24,15,11,6,0,224.750,"$95,345.00"
5,2015,20,15,11,11,2,225.400,"$178,542.00"
6,2014,18,17,6,4,3,226.710,"$163,778.00"
7,2013,26,21,13,10,2,228.810,"$186,464.59"
8,2011,13,12,9,8,4,226.420,"$137,375.00"
9,2010,12,10,9,4,0,218.820,"$62,950.00"


In [10]:
df.Years = df.Years.astype(int)
df.Events = df.Events.astype(int)
df.Cashes = df.Cashes.astype(int)
#df.Match Play.astype(int)
df.CRA = df.CRA.astype(int)
df.Titles = df.Titles.astype(int)
df.AVG = df.AVG.astype(float)
#df.Earnings.astype(float)

In [11]:
df[df.CRA == 3]

,Years,Events,Cashes,Match Play,CRA,Titles,AVG,Earnings
2,2018,14,10,6,3,2,221.38,"$110,500.00"
10,2009,16,15,10,3,0,217.94,"$80,090.00"


In [12]:
df[df.Cashes == 15]

,Years,Events,Cashes,Match Play,CRA,Titles,AVG,Earnings
3,2017,15,15,11,7,4,229.39,"$238,912.00"
4,2016,24,15,11,6,0,224.75,"$95,345.00"
5,2015,20,15,11,11,2,225.40,"$178,542.00"
10,2009,16,15,10,3,0,217.94,"$80,090.00"


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Years       12 non-null     int64  
 1   Events      12 non-null     int64  
 2   Cashes      12 non-null     int64  
 3   Match Play  12 non-null     object 
 4   CRA         12 non-null     int64  
 5   Titles      12 non-null     int64  
 6   AVG         12 non-null     float64
 7   Earnings    12 non-null     object 
dtypes: float64(1), int64(5), object(2)
memory usage: 896.0+ bytes


In [14]:
for row in df[df.Cashes.isin([15,17])].itertuples():
    #print(row[1:])
    pass

In [15]:
row[0:]

(10, 2009, 16, 15, '10', 3, 0, 217.94, '$80,090.00')

In [16]:
len(row[1:])

8

In [17]:
db     = sqlite3.connect(":memory:")
cursor = db.cursor()
cursor.execute("""
 CREATE TABLE JASONBELMONTE(YEARS, EVENTS, CASHES, MATCHPLAY, CRA, TITLES, AVG, EARNINGS)
""")

for row in df[df.Cashes.isin([15,17])].itertuples():
    insert_sql_syntax = """
    INSERT INTO JASONBELMONTE(YEARS, EVENTS, CASHES, MATCHPLAY, CRA, TITLES, AVG, EARNINGS)
    VALUES (?,?,?,?,?,?,?,?)
    """
    cursor.execute(insert_sql_syntax, row[1:])
    #print(row[1:])
db.commit()

#db.close()

In [18]:
cursor.execute("""
    SELECT *
    FROM JASONBELMONTE
    WHERE CASHES == 15
""")

In [19]:
for row in cursor.execute("""
    SELECT *
    FROM JASONBELMONTE
    WHERE CASHES == 15
"""):
    print(row)

(2017, 15, 15, '11', 7, 4, 229.39, '$238,912.00')
(2016, 24, 15, '11', 6, 0, 224.75, '$95,345.00')
(2015, 20, 15, '11', 11, 2, 225.4, '$178,542.00')
(2009, 16, 15, '10', 3, 0, 217.94, '$80,090.00')
